In [1]:
# Importation des bibliothèques nécessaires
import mlflow
from mlflow.models import infer_signature

import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [2]:
# Configuration de l'URI de suivi pour MLflow
mlflow.set_tracking_uri(uri="http://127.0.0.1:8085")

In [3]:
# Charger le jeu de données depuis un fichier CSV
data = pd.read_csv('./Data/X_train.csv')
# Afficher les premières lignes du jeu de données
data.head()

,sk_id_curr,region_rating_client,ext_source_2,taille_famille,age_client,anciennete_entreprise,anciennete_cni,anciennete_portable,year_registration,heure_demande_pret,...,flag_work_phone_Oui,flag_phone_Oui,reg_region_not_live_region_adresse_identique,reg_region_not_work_region_adresse_identique,reg_city_not_live_city_adresse_identique,reg_city_not_work_city_adresse_identique,live_city_not_work_city_adresse_identique,flag_document_3_Oui,flag_document_6_Oui,flag_document_8_Oui
0,100002,1,0.262949,0,0,1,1,3,1,0,...,False,True,True,True,True,True,True,True,False,False
1,100003,0,0.622246,1,2,2,0,2,0,0,...,False,True,True,True,True,True,True,True,False,False
2,100004,1,0.555912,0,3,1,1,2,2,0,...,True,True,True,True,True,True,True,False,False,False
3,100006,1,0.650442,1,3,3,1,2,4,1,...,False,False,True,True,True,True,True,True,False,False
4,100007,1,0.322738,0,3,3,1,3,2,0,...,False,False,True,True,True,False,False,False,False,True


In [4]:
# Séparer les caractéristiques (X) de la cible (y)
X = data.drop('sk_id_curr', axis=1)
y = data['sk_id_curr']

In [5]:
# Division des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Définition des hyperparamètres du modèle
params = {"n_estimators": 100, "random_state": 101, "max_depth": 2, "n_jobs": 4}

# Entraînement du modèle
rf = RandomForestClassifier(**params)
rf.fit(X_train, y_train)

# Prédiction sur l'ensemble de test
y_pred = rf.predict(X_test)

# Calcul de la précision comme une métrique de perte cible
accuracy = accuracy_score(y_test, y_pred)

In [6]:
# Afficher les dimensions des ensembles d'entraînement et de test
print("Dimensions de l'ensemble d'entraînement X :", X_train.shape)
print("Dimensions de l'ensemble de test X :", X_test.shape)

Dimensions de l'ensemble d'entraînement X : (35250, 59)
Dimensions de l'ensemble de test X : (8813, 59)


In [7]:
# Définition de l'expérience MLflow pour le modèle RandomForest
mlflow.set_experiment("MLflow RandomForest_model")

2023/12/09 15:08:53 INFO mlflow.tracking.fluent: Experiment with name 'MLflow RandomForest_model' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/707525579124339687', creation_time=1702130933638, experiment_id='707525579124339687', last_update_time=1702130933638, lifecycle_stage='active', name='MLflow RandomForest_model', tags={}>

In [8]:
# Démarrer une exécution MLflow
with mlflow.start_run():
    # Enregistrer les hyperparamètres
    mlflow.log_params(params)

    # Enregistrer la métrique de perte (dans ce cas, la précision)
    mlflow.log_metric("accuracy", accuracy)

    # Définir une étiquette que l'on peut utiliser pour se rappeler le but de cette exécution
    mlflow.set_tag("Training Info", "Modèle de base RandomForestClassifier pour les données")

    # Inférer la signature du modèle
    signature = infer_signature(X_train, rf.predict(X_train))

    # Enregistrer le modèle
    model_info = mlflow.sklearn.log_model(
        sk_model=rf,
        artifact_path="data_model",
        signature=signature,
        input_example=X_train,
        registered_model_name="tracking-RandomForest_model",
    )

/Users/admin/anaconda3/lib/python3.11/site-packages/mlflow/models/signature.py:212: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  inputs = _infer_schema(model_input) if model_input is not None else None
/Users/admin/anaconda3/lib/python3.11/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing 

In [9]:
# Charger le modèle enregistré à partir de l'URI du modèle
loaded_model = mlflow.pyfunc.load_model(model_info.model_uri)

In [11]:
# Effectuer des prédictions sur l'ensemble de test à l'aide du modèle chargé
predictions = loaded_model.predict(X_test)

# Obtenir les noms des caractéristiques (features) à partir des données
feature_names = X_test.columns

# Convertir les données de caractéristiques de validation X_test en un DataFrame Pandas
result = pd.DataFrame(X_test, columns=feature_names)

# Ajouter les classes réelles au DataFrame
result["actual_class"] = y_test

# Ajouter les prédictions du modèle au DataFrame
result["predicted_class"] = predictions

# Afficher les quatre premières lignes du résultat
result[:4]

,region_rating_client,ext_source_2,taille_famille,age_client,anciennete_entreprise,anciennete_cni,anciennete_portable,year_registration,heure_demande_pret,pop_region,...,reg_region_not_live_region_adresse_identique,reg_region_not_work_region_adresse_identique,reg_city_not_live_city_adresse_identique,reg_city_not_work_city_adresse_identique,live_city_not_work_city_adresse_identique,flag_document_3_Oui,flag_document_6_Oui,flag_document_8_Oui,actual_class,predicted_class
5343,1,0.452372,1,3,4,1,3,4,1,3,...,True,True,True,True,True,False,False,False,106246,100433
10307,1,0.236627,0,0,3,2,3,1,1,1,...,True,True,True,True,True,True,False,False,111994,119248
42362,1,0.719790,1,0,1,1,3,0,1,3,...,True,True,True,True,True,True,False,False,149033,108085
12384,1,0.322074,1,1,3,2,0,4,1,1,...,True,True,True,True,True,True,False,False,114436,100433
